In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### This Online Retail II data set contains all the transactions occurring for a UK-based and registered, non-store online retail between 01/12/2009 and 09/12/2011.The company mainly sells unique all-occasion gift-ware.

### If you want to have detailed information about the features you can visit: https://archive.ics.uci.edu/ml/datasets/Online+Retail+IIHere 

### Our main goal is to cluster all the customers according to their attributes and gain more information about various customer patterns.

# Importing Libraries 

In [3]:
# for data manipulation and analysis
import pandas as pd
import numpy as np

# for plotting
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
sns.set_style('darkgrid')

# Silhouette analysis
from sklearn.metrics import silhouette_score

# To perform KMeans clustering 
from sklearn.cluster import KMeans

# for scaling
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

#### In order to import an exel file we need to install this libraries :

In [4]:
pip install openpyxl

In [5]:
pip install xlrd

# 0) Importing Dataset
#### loading the dataset :

In [6]:
df = pd.read_excel('/kaggle/input/online-retail-data-set-from-uci-ml-repo/Online Retail.xlsx')

# looking at top 5 rows
df.head()

# 1) Preprocessing 

### Dataset Overview :

In [7]:
# checking the number of rows and columns
df.shape

In [8]:
# looking at the overall picture
df.info()

#### Checking missing values :

In [9]:
# checking the number of missing values in each column
df.isnull().sum()

As its shown there are many null data in CustomerID and Descripton columns

### **Basic Cleaning**

#### Cleaning duplicated data :

Rows which belong to same customers and transactions 

In [10]:
# count of duplicated rows in the data
df.duplicated().sum()

In [11]:
# removing the duplicate rows
df = df[~df.duplicated()]
df.shape

#### Cleaning transactions with negetive quantity:

these are the transactions that have negative quantity which indicates returned or cancelled orders
- Cancelled Ones: starting InvoiceNo with C

In [12]:
df[df['InvoiceNo'].str.startswith('C')==True]

In [13]:
# removing all the invoice number who starts with 'C' as they are returned orders
df = df[df['InvoiceNo'].str.startswith('C')!=True]
df.shape

In [14]:
# checking the number of unique transactions
# though there are more than 5 lakh entries but the number of transaction happened is 21892
df.InvoiceNo.nunique()

In [15]:
# checking the unique stock ids in the data or number of unqiue item sold by retailer
df.StockCode.nunique()

In [16]:
# top 10 stock ids that sold the most
df.StockCode.value_counts().head(10)

- Negetive Qauntity :

In [17]:
# looking at the distribution of the quantity
# we seen that there is negative value which might indicate return orders
df.Quantity.describe()

In [18]:
# looking at the data where quantity is negative and possible explanation is these are return orders or cancelled order
df[df['Quantity']<0]

In [19]:
# keeping only those transactions that have successfully ordered
df = df[df['Quantity']>=0]
df.shape

In [20]:
df

In [21]:
print('The minimum date is:',df.InvoiceDate.min())
print('The maximum date is:',df.InvoiceDate.max())

In [22]:
# checking the distribution of unit price
df.UnitPrice.describe(percentiles=[0.25,0.5,0.75,0.9,0.95,0.99])

In [23]:
# we see that more than 90% have country as UK which is obvious as the retailer is UK based
df.Country.value_counts(normalize=True)

In [24]:
# putting UK as one country and combine rest countries into one category
df['Country'] = df['Country'].apply(lambda x:'United Kingdom' if x=='United Kingdom' else 'Others')
df.Country.value_counts(normalize=True)

In [25]:
df

In [26]:
# checking the number of unique item list
df.Description.nunique()

In [27]:
# top 10 item sold
df.Description.value_counts().head(10)

In [28]:
# there are cases where the descriptions contains some code/name which are not directly refers to sales
# checking the data where description = ? and it is noted that customerid is NaN and unit price is 0
df[df['Description'].str.startswith('?')==True]

#### As it's shown these data is unusable so we gonna remove them from dataset!

In [29]:
# removing all the above entries
df = df[df['Description'].str.startswith('?')!=True]
df.shape

#### Data with starting description with * which has NAN customerID but is does not effect on our analysis  ( because customerID is something private and does not effect on grouping )

In [30]:
# checking the data where description = * and it is noted that customerid is NaN
df[df['Description'].str.startswith('*')==True]

In [31]:
# replacing with appropriate name
df['Description'] = df['Description'].replace(('*Boombox Ipod Classic','*USB Office Mirror Ball'),
                                             ('BOOMBOX IPOD CLASSIC','USB OFFICE MIRROR BALL'))

In [32]:
# Description have actual entries in uppercase words and those who don't have are some of the noises in the dataset
df[df['Description'].str.islower()==True]['Description'].value_counts()

In [33]:
# removing all the above noises
df = df[df['Description'].str.islower()!=True]
df.shape

In [34]:
# Description have actual entries in uppercase words and those who don't have are some of the noises in the dataset
df[df['Description'].str.istitle()==True]['Description'].value_counts()

In [35]:
# removing all the above listed noises
df = df[df['Description'].str.istitle()!=True]
df.shape

In [36]:
df['Description'] = df['Description'].str.strip()

### Customer ID checking...

In [37]:
# count of unique customer
df.CustomerID.nunique()

In [38]:
# checking where customer id is null
df[df.CustomerID.isnull()]

In [39]:
# removing entries where customer id is null
df = df[~df.CustomerID.isnull()]
df.shape

In [40]:
df.info()

In [41]:
df

 Checking if there is any null data left :

In [42]:
df.isnull().sum()

In [43]:
# checking random 5 rows from data
df.sample(5)

## 2) Visualisation 

#### creating some columns for understandable exploratory...

Firstly we will create a new column as total value according to the unit price and quantity purchased.

In [44]:
df['Amount'] = df['Quantity']*df['UnitPrice']
df['year'] = df['InvoiceDate'].dt.year
df['month'] = df['InvoiceDate'].dt.month
df['day'] = df['InvoiceDate'].dt.day
df['hour'] = df['InvoiceDate'].dt.hour
df['day_of_week'] = df['InvoiceDate'].dt.dayofweek

In [45]:
df

In [46]:
column = ['InvoiceNo','Amount']

plt.figure(figsize=(15,5))
for i,j in enumerate(column):
    plt.subplot(1,2,i+1)
    sns.barplot(x = df[df['Country']=='United Kingdom'].groupby('Description')[j].nunique().sort_values(ascending=False).head(10).values,
                y = df[df['Country']=='United Kingdom'].groupby('Description')[j].nunique().sort_values(ascending=False).head(10).index,
                color='yellow')
    plt.ylabel('')
    if i==0:
        plt.xlabel('Sum of quantity')
        plt.title('Top 10 products purchased by customers in UK',size=15)
    else:
        plt.xlabel('Total Sales')
        plt.title('Top 10 products with most sales in UK',size=15)
        
plt.tight_layout()
plt.show()

In [47]:
column = ['Others','United Kingdom']

plt.figure(figsize=(15,5))
for i,j in enumerate(column):
    plt.subplot(1,2,i+1)
    sns.barplot(x = df[df['Country']==j].groupby('Description')['UnitPrice'].mean().sort_values(ascending=False).head(10).values,
                y = df[df['Country']==j].groupby('Description')['UnitPrice'].mean().sort_values(ascending=False).head(10).index,
                color='yellow')
    plt.ylabel('')
    if i==0:
        plt.xlabel('Unit Price')
        plt.title('Top 10 high value products outside UK',size=15)
    else:
        plt.xlabel('Unit Price')
        plt.title('Top 10 high value products in UK',size=15)
        
plt.tight_layout()
plt.show()


In [48]:
# Looking the distribution of column Quantity
plt.figure(figsize=(10,7))

skewness = round(df.Quantity.skew(),2)
kurtosis = round(df.Quantity.kurtosis(),2)
mean = round(np.mean(df.Quantity),0)
median = np.median(df.Quantity)

plt.subplot(2,2,1)
sns.boxplot(y=df.Quantity)
plt.title('Boxplot\n Mean:{}\n Median:{}\n Skewness:{}\n Kurtosis:{}'.format(mean,median,skewness,kurtosis))

plt.subplot(2,2,2)
sns.boxplot(y=df[df.Quantity<5000]['Quantity'])
plt.title('Distribution when Quantity<5000')

plt.subplot(2,2,3)
sns.boxplot(y=df[df.Quantity<200]['Quantity'])
plt.title('Distribution when Quantity<200')

plt.subplot(2,2,4)
sns.boxplot(y=df[df.Quantity<50]['Quantity'])
plt.title('Distribution when Quantity<50')

plt.show()

In [49]:
# removing the expectional case where quantity > 70000
df = df[df['Quantity']<70000]

In [50]:
# Looking the distribution of column Unit Price
plt.figure(figsize=(10,7))

skewness = round(df.UnitPrice.skew(),2)
kurtosis = round(df.UnitPrice.kurtosis(),2)
mean = round(np.mean(df.UnitPrice),0)
median = np.median(df.UnitPrice)

plt.subplot(2,2,1)
sns.boxplot(y=df.UnitPrice)
plt.title('Boxplot\n Mean:{}\n Median:{}\n Skewness:{}\n Kurtosis:{}'.format(mean,median,skewness,kurtosis))

plt.subplot(2,2,2)
sns.boxplot(y=df[df.UnitPrice<300]['UnitPrice'])
plt.title('Distribution when Unit Price<300')

plt.subplot(2,2,3)
sns.boxplot(y=df[df.UnitPrice<50]['UnitPrice'])
plt.title('Distribution when Unit Price<50')

plt.subplot(2,2,4)
sns.boxplot(y=df[df.UnitPrice<10]['UnitPrice'])
plt.title('Distribution when Unit Price<10')

plt.show()

In [51]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
sns.barplot(y = df[df['Country']=='United Kingdom'].groupby('CustomerID')['Amount'].sum().sort_values(ascending=False).head(10).values,
            x = df[df['Country']=='United Kingdom'].groupby('CustomerID')['Amount'].sum().sort_values(ascending=False).head(10).index, 
            color='green')
plt.ylabel('Sales')
plt.xlabel('Customer IDs')
plt.xticks(rotation=45)
plt.title('Top 10 customers in terms of sales in UK',size=15)

plt.subplot(1,2,2)
sns.barplot(y = df[df['Country']=='United Kingdom'].groupby('CustomerID')['InvoiceNo'].nunique().sort_values(ascending=False).head(10).values,
            x = df[df['Country']=='United Kingdom'].groupby('CustomerID')['InvoiceNo'].nunique().sort_values(ascending=False).head(10).index, 
            color='green')
plt.ylabel('Number of visits')
plt.xlabel('Customer IDs')
plt.xticks(rotation=45)
plt.title('Top 10 customers in terms of frequency in UK',size=15)

plt.show()

In [52]:
plt.figure(figsize=(12,5))
df[df['Country']=='United Kingdom'].groupby(['year','month'])['Amount'].sum().plot(kind='line',label='UK',color='blue')
df[df['Country']=='Others'].groupby(['year','month'])['Amount'].sum().plot(kind='line',label='Other',color='grey')
plt.xlabel('Year-Month',size=12)
plt.ylabel('Total Sales', size=12)
plt.title('Sales in each month for an year', size=15)
plt.legend(fontsize=12)
plt.show()

In [53]:
plt.figure(figsize=(12,5))
df[df['Country']=='United Kingdom'].groupby(['day'])['Amount'].sum().plot(kind='line',label='UK',color='blue')
df[df['Country']=='Others'].groupby(['day'])['Amount'].sum().plot(kind='line',label='Other',color='grey')
plt.xlabel('Day',size=12)
plt.ylabel('Total Sales', size=12)
plt.title('Sales on each day of a month', size=15)
plt.legend(fontsize=12)
plt.show()

In [54]:

plt.figure(figsize=(12,5))
df[df['Country']=='United Kingdom'].groupby(['hour'])['Amount'].sum().plot(kind='line',label='UK',color='blue')
df[df['Country']=='Others'].groupby(['hour'])['Amount'].sum().plot(kind='line',label='Other',color='grey')
plt.xlabel('Hours',size=12)
plt.ylabel('Total Sales', size=12)
plt.title('Sales in each hour in a day', size=15)
plt.legend(fontsize=12)
plt.show()

## 3) Modeling 

We will now create a new Dataframe which will have each unique customer will total amount they have spent.In this way we can cluster them according to Amount spent.

In [55]:
each_customer = df.groupby('CustomerID')['Amount'].sum()

In [56]:
each_customer = each_customer.reset_index()

In [57]:
each_customer

#### Note:We are take .sum() to add up all the amounts and get a total for each CustomerID.



#### We can also find out how many times a Customer has visited the store by the column Invoice.

In [58]:
each_customer_freq = df.groupby('CustomerID')['InvoiceNo'].count()

In [59]:
each_customer_freq = each_customer_freq.reset_index()

In [60]:
each_customer_freq

#### Note:Here we have taken ‘Count’ of each invoices for each customer so that we can get a total number of invoices for each customer.

#### Lets merge the two dataframes:

In [61]:
customer_details = pd.merge(each_customer_freq,each_customer,on='CustomerID')

In [62]:
customer_details

Here comes the interesting part:

We can also further get more details like which customers have not visited store for a long time and which have.By this we can cluster Customers which have not visited and see the underlying pattern.

This all can be done by the column ‘InvoiceDate’ which contains the datetime with respect to each invoice.

First we need to make sure that the columns is in datetime format ,if not we need to convert it to datetime format.



In [63]:
df.dtypes

As we can see the ‘InvoiceDate’ column is in datetime format.

Now as we need the exact no of days a customer has been last active we will use the InvoiceDate column to find out the most recent date from the InvoiceDate column and then subtract the Maximum date with the Invoice date for each customer which will give us no. of days the customer was last active.

In [64]:
max_date = max(df['InvoiceDate'])

In [65]:
max_date

In [66]:
df['recency']=max_date-df['InvoiceDate']

In [67]:
df.head()

#### Now we can groupby each customer and take out the .min() from the recency column to get the days when the customer was last seen.

In [68]:
dates = df.groupby('CustomerID')['recency'].min()

In [69]:
dates = dates.reset_index()

In [70]:
import datetime as dt

In [71]:
dates['recency']=dates['recency'].dt.days

In [72]:
dates

#### Above we have used dt.days of the datetime library to get only the days and remove the time.

#### Now lets merge this Dataframe with the earlier ‘Customer_details’ Dataframe.

In [73]:
customer_details = pd.merge(customer_details,dates,on='CustomerID')

In [74]:
customer_details

#### Now the ‘Amount’ column has lots of outliers which we need to remove in order to form good clusters.

In [75]:
z = customer_details.Amount.quantile(0.05)

In [76]:
y = customer_details.Amount.quantile(0.95)

In [77]:
iqr=y-z

Here the quantile function is very important,__what it does is it gives us the value for the first 5% of the Distribution on data in Amount column and same for 95%.(0.95).The value which we get shows that 5% & 95 % of data in our column is less than(5%) and greater than(95%).__

We can get the interquartile range for the above quantile functions by subtracting 95 percentile with 5 percentile.

In [78]:
customer_details = customer_details[(customer_details['Amount']>=z-1.5*iqr) & (customer_details['Amount']<=y+1.5*iqr)]

In [79]:
customer_details

Here we have used the quartile values to __remove the outliers from the Amount column ,anything which is below ‘z-1.5*iqr’ and anything above ‘y+1.5*iqr’ will be removed.This is the standard way to remove outliers__.Repeating these steps for other features.

In [80]:
q1 = customer_details.InvoiceNo.quantile(0.05)
q3 = customer_details.InvoiceNo.quantile(0.95)
iqr = q3 - q1
customer_details = customer_details[(customer_details['recency']>=q1-1.5*iqr) & (customer_details['recency']<=q3+1.5*iqr)]

In [81]:
customer_details

#### Once we have removed the outliers we can scale the data as the scaling standards differ alot between the columns.

In [82]:
from sklearn.preprocessing import StandardScaler

In [83]:
scc = StandardScaler()

In [84]:
scaled = scc.fit_transform(customer_details[['InvoiceNo','Amount','recency']])


In [85]:
scaled.shape

In [86]:
scaled_values = pd.DataFrame(scaled)

In [87]:
scaled_values.columns=['InvoiceNo','Amount','recency']

In [88]:
scaled_values

##  K-means Clustering

#### Now as we know in Kmeans cluster we need to provide ‘K’ value for the algorithm as the no of clusters.We can find out the optimal value for K with elbow graph.

In [89]:
from sklearn.cluster import KMeans

In [90]:
scores=[]
for i in range(2,7):
    kmeans = KMeans(n_clusters=i,max_iter=40,verbose=True).fit(scaled_values)
    scores.append(kmeans.inertia_)

In [91]:
len(scores)

In [92]:
plt.plot(range(2,7,1),scores)
plt.xticks(ticks=range(2,7))
plt.show()

### Now we should use a metric to evaluate the results:

### Calinski-Harabasz Index

The Calinski-Harabasz index also known as the Variance Ratio Criterion, is the ratio of the sum of between-clusters dispersion and of inter-cluster dispersion for all clusters, __the higher the score , the better the performances.__

The score is defined as the ratio between the within-cluster dispersion and the between-cluster dispersion. The C-H Index is a great way to evaluate the performance of a Clustering algorithm as it does not require information on the ground truth labels.

In [93]:
from sklearn.metrics import calinski_harabasz_score
for i in range(2,7):
    kmeans = KMeans(n_clusters=i,max_iter=40).fit(scaled_values)
    cluster_labels=kmeans.labels_
    cal = calinski_harabasz_score(scaled_values,cluster_labels)
    print('For n_cluster{}  Calinski score is {}'.format(i,cal))
    

From the above elbow graphs and silhouette scores we can take ‘3’ as our k value for number of clusters.

Lets create our final Model:

In [94]:
kmeans = KMeans(n_clusters=3,max_iter=50).fit(scaled_values)
clusters = kmeans.labels_

In [95]:
customer_details['Cluster']=clusters

In [96]:
customer_details

Here we have created our final model with 3 clusters and added our cluster labels obtained from ‘kmeans.labels_’ to our Dataframe consisting of Unique customers.

With the help of boxplots we can visualize the clusters formed on different features:

In [97]:
sns.boxplot(x='Cluster',y='Amount',data=customer_details)
plt.show()

In [98]:
sns.boxplot(x='Cluster',y='recency',data=customer_details)
plt.show()

In [99]:
sns.boxplot(x='Cluster',y='InvoiceNo',data=customer_details)
plt.show()

### Sources : 
- https://medium.com/@mayureshrpalav/clustering-customers-online-retail-dataset-516e961e7bc
- https://www.kaggle.com/code/mittalvasu95/cohort-rfm-k-means#III.-k-Means-Clustering